In [1]:
!pip install tensorflow==2.3.0
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()
tf.__version__

ImportError: cannot import name 'GceClusterResolver' from 'tensorflow.python.distribute.cluster_resolver.gce_cluster_resolver' (C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\distribute\cluster_resolver\gce_cluster_resolver.py)

In [2]:
def grad(hypothesis, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(hypothesis, labels)
    return tape.gradient(loss_value, [W, b])

In [3]:
def expotential_decay(epoch):
    starter_rate = 0.01
    k = 0.96
    exp_rate = starter_rate * exp(-k*t)
    return exp_rate

In [4]:
# Standardization = (data - np.mean(data)) / np.sqrt(np.sum((data - np.mean(data))^2) / np.count(data))

# Normalization = (data - np.min(data, 0)) / (np.max(data, 0) - np.min(data, 0))

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
# pca.fit(X)
# X = pca.transform(X)

In [6]:
# L2_loss = tf.nn.l2_loss(w)

In [7]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
              [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
              [819.929993, 824.400021, 1438100, 818.97998, 824.159973],
              [816, 820.958984, 1008100, 815.48999, 819.23999],
              [819.359985, 823, 1188100, 818.469971, 818.97998],
              [819, 823, 1198100, 816, 820.450012],
              [811.700012, 815.25, 1098100, 809.780029, 813.669983],
              [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

def normalization(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    
    return numerator / denominator

xy = normalization(xy)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random_normal([4, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_normal([1]), dtype=tf.float32)

def linearReg_fn(features):
    hypothesis = tf.matmul(features, W) + b

    return hypothesis

def l2_loss(loss, beta = 0.01):
    W_reg = tf.nn.l2_loss(W)
    loss = tf.reduce_mean(loss + W_reg * beta)

    return loss

def loss_fn(hypothesis, labels, flag=False):
    cost = tf.reduce_mean(tf.square(hypothesis - labels))
    if(flag):
        cost = l2_loss(cost)
    
    return cost

AttributeError: module 'tensorflow' has no attribute 'random_normal'

In [9]:
is_decay = True
starter_learning_rate = 0.1

if (is_decay):
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 50, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
else:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=starter_learning_rate)

def grad(features, labels, l2_flag):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(linearReg_fn(features), labels, l2_flag)
    
    return tape.gradient(loss_value, [W, b]), loss_value

for step in range(101):
    for features, labels in tf.contrib.enable_eager_execution.Iterator(dataset):
        features = tf.cast(labels, tf.float32)
        grads, loss_value = grad(linearReg_fn(features), features, labels, False)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]), global_step=global_step)
        if step % 10 == 0:
            print("Iter: {}, Loss: {:.4f}, Learning Rate: {:.8f}".format(step, loss_value, optimizer._learning_rate()))

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'exponential_decay'

In [13]:
import keras
keras.__version__

'2.4.3'

In [14]:
fasion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (train_images, test_labels) = fasion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='ada,',
loss = 'sparse_categorical_crossentropy',
metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)
test_loss, test_acc = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)
np.argmax(predictions[0])